### doc_topics
ESTABELECE RELAÇÕES ENTRE DOCUMENTOS E TÓPICOS NA BASE DE DADOS NO MYSQL. TRABALHAMOS COM O ACERVO **ANTONIO AZEREDO DA SILVEIRA, MINISTÉRIO DAS RELAÇÕES EXTERIORES**.

In [1]:
import nltk
import os
import codecs
import string
import matplotlib.pyplot as plt
import matplotlib as mpl
from gensim import corpora, models, similarities #Latent Dirichlet Allocation implementation with Gensim
import pyLDAvis
import pyLDAvis.gensim
from IPython.display import clear_output

import getpass
from sshtunnel import SSHTunnelForwarder
import pymysql

C:\Anaconda3\lib\site-packages\funcy\decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
C:\Anaconda3\lib\site-packages\funcy\decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
C:\Anaconda3\lib\site-packages\funcy\decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
C:\Anaconda3\lib\site-packages\funcy\decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
C:\Anaconda3\lib\site-packages\funcy\decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
C:\Anaconda3\lib\site-packages\funcy\decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use 

In [18]:
outputs = "../outputs/"

In [2]:
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
Seleciona textos da base mysql para fazer modelagem de tópicos
NOTA: DEIXEI EM RAW PORQUE ESSA EXTRAÇÃO TORNA O CÓDIGO MUITO LENTO, MAS DEVE SER FEITO DESSA FORMA NO CPU DO RENATO
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
count = 0
texts = []
percentil = 10500/100
pass_sshkey = getpass.getpass()
pass_mysql = getpass.getpass()

with SSHTunnelForwarder(('200.20.164.147', 22),
                        ssh_private_key = "C:/Users/marcelo.ribeiro/Dropbox/A-Marcelo/Educação-Trabalho/2016-CPDOC/Remoto/marcelo_priv_rsa",
                        #ssh_private_key = '/home/rsouza/.ssh/id_rsa',
                        ssh_private_key_password = pass_sshkey,
                        #ssh_private_key_password = input('Entre com a senha da chave privada'),
                        ssh_username="marcelobribeiro",
                        #ssh_username="rsouza",
                        remote_bind_address=('127.0.0.1', 3306)) as server:
    
    conn = pymysql.connect(host='localhost', 
                           port=server.local_bind_port, 
                           user='marcelobribeiro', 
                           #user='rsouza',
                           passwd=pass_mysql,
                           #passwd=input('Entre com a senha do usuário no banco de dados'),
                           db='CPDOC_AS',
                           use_unicode=True, 
                           charset="utf8")
    cur = conn.cursor()
    
    '''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
    captura documentos da tabela docs para posterior modelagem de tópicos
    '''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
    cur.execute("SELECT * FROM CPDOC_AS.docs WHERE main_language = 1") # seleciona apenas textos majoritariamente em português    
    row = cur.fetchone()
                    
    while row is not None:
        count += 1
        row = cur.fetchone()
    
        if count % percentil == 0: 
            clear_output()
            print(int(count/percentil),'% done')
                
        text =  row[1]
        text = text.split()
        symbols = [x for x in string.punctuation]
        text = [p for p in text if p not in symbols]
        text = [p.strip(string.punctuation) for p in text]
        text = [p for p in text if not p.isdigit()]
        text = [p for p in text if len(p)>1]
        texts.append(text)
        
        #if count == 3: break #amostra de apenas 3 linhas. No computador do Renato, não usar o break.

········
········


In [ ]:
# load nltk's English stopwords as variable called 'stopwords'

additional_words = ['mr','one', 'two', 'three', 'four', 
                    'five', 'um', 'dois', 'três', 'quatro', 
                    'cinco', 'janeiro', 'fevereiro', 'março', 
                    'abril', 'maio', 'junho', 'julho', 'agosto', 
                    'setembro', 'outubro', 'novembro', 'dezembro', 
                    'january', 'february', 'march', 'april', 'may', 
                    'june', 'july', 'august', 'september', 
                    'october', 'november', 'december', 'países', 
                    'ser', 'país', 'ainda', 'milhões', 'maior', 
                    'anos', 'grande', 'apenas', 'outros', 'pode', 
                    'parte', 'partes', 'item', 'vossa', 'nota', 
                    'havia', 'pt', 'vg', 'ptvg', 'eh', 'nr', 'hrs', 
                    'pais', 'parte', 'hoje', 'brasemb', 'ontem', 
                    'dia', 'countries', 'would', 'new', 'also', 
                    'must', 'draft', 'shall', 'item', 'page', 
                    'th', 'anos', 'ii', 'dias', 'poderá', 'caso', 
                    'casos', 'qualquer', 'ano', 'mil', 'pessoas', 
                    'único', 'única', 'únicos', 'únicas', 'índice', 
                    'expedido', 'co', 'mm', 'er', 'via', 'ww', 'ra', 
                    'ia', 'ca', 'nu', 'wa', 'aa', 'ms', 'dc', 'mmm', 'pa']

stopwords = nltk.corpus.stopwords.words('english') + \
            nltk.corpus.stopwords.words('portuguese') + \
            nltk.corpus.stopwords.words('french') + \
            nltk.corpus.stopwords.words('spanish') + \
            additional_words

stopwords = list(set(stopwords))

### Latent Dirichlet Allocation

In [7]:
#%time texts = [[word for word in text] for text in texts]
%time texts = [[word for word in text if word not in stopwords] for text in textos]

Wall time: 0 ns


In [8]:
#print(len([word for word in texts[0] if word not in stopwords]))
print(len(texts[0]))

54


In [9]:
dictionary = corpora.Dictionary(texts)
dictionary.filter_extremes(no_below=1, no_above=0.8)
#print(dictionary.token2id['']) #examina o id a partir do token
#print(dictionary.id2token[0]) #examina o token a partir do id
dictionary.filter_tokens(bad_ids=[0,]) #retira palavras a partir do id
corpus = [dictionary.doc2bow(text) for text in texts]

In [10]:
len(corpus)

3

In [11]:
%time lda = models.LdaModel(corpus, num_topics=50, id2word=dictionary, passes=10)

Wall time: 668 ms


In [12]:
lda.print_topics(-1, num_words=20)

[(0,
  '0.003*da + 0.003*grupo + 0.003*direitos + 0.003*humanos + 0.003*comunicações + 0.003*que + 0.003*por + 0.002*comissão + 0.002*no + 0.002*dos + 0.002*brasil + 0.002*ao + 0.002*foram + 0.002*se + 0.002*contra + 0.002*américa + 0.002*embaixador + 0.002*as + 0.002*membro + 0.002*discriminação'),
 (1,
  '0.002*pas + 0.002*lizou + 0.002*julho + 0.002*pela + 0.002*natalicia + 0.002*pronunciou + 0.002*referente + 0.002*sendo + 0.002*precederam + 0.002*afirmara + 0.002*beverly + 0.002*respeitosos + 0.002*tocante + 0.002*vista + 0.002*interesses + 0.002*foi + 0.002*apreço + 0.002*aos + 0.002*ação + 0.002*embora'),
 (2,
  '0.003*da + 0.003*grupo + 0.003*humanos + 0.003*no + 0.003*que + 0.003*dos + 0.003*comunicações + 0.003*contra + 0.003*comissão + 0.003*brasil + 0.003*foram + 0.003*direitos + 0.003*junto + 0.003*alegações + 0.003*as + 0.002*se + 0.002*ao + 0.002*realizada + 0.002*na + 0.002*membro'),
 (3,
  '0.003*que + 0.003*da + 0.003*bolívia + 0.003*estados + 0.003*dos + 0.003*na + 0

In [13]:
lda.show_topics(num_topics=-1)

[(0,
  '0.003*da + 0.003*grupo + 0.003*direitos + 0.003*humanos + 0.003*comunicações + 0.003*que + 0.003*por + 0.002*comissão + 0.002*no + 0.002*dos'),
 (1,
  '0.002*tocante + 0.002*vista + 0.002*afirmara + 0.002*beverly + 0.002*respeitosos + 0.002*pas + 0.002*apreço + 0.002*foi + 0.002*aos + 0.002*interesses'),
 (2,
  '0.003*da + 0.003*grupo + 0.003*humanos + 0.003*no + 0.003*que + 0.003*dos + 0.003*comunicações + 0.003*contra + 0.003*comissão + 0.003*brasil'),
 (3,
  '0.003*que + 0.003*da + 0.003*bolívia + 0.003*estados + 0.003*dos + 0.003*na + 0.003*unidos + 0.003*os + 0.002*não + 0.002*um'),
 (4,
  '0.002*da + 0.002*no + 0.002*humanos + 0.002*comissão + 0.002*grupo + 0.002*que + 0.002*direitos + 0.002*brasil + 0.002*por + 0.002*ao'),
 (5,
  '0.002*tocante + 0.002*vista + 0.002*afirmara + 0.002*beverly + 0.002*respeitosos + 0.002*pas + 0.002*apreço + 0.002*foi + 0.002*aos + 0.002*interesses'),
 (6,
  '0.003*da + 0.003*que + 0.003*no + 0.003*unidos + 0.003*dos + 0.003*não + 0.003*hum

In [14]:
topics_matrix = lda.show_topics(formatted=False, num_words=20, num_topics=-1)
topics_matrix[34]

(34,
 [('pas', 0.0024213075060532689),
  ('lizou', 0.0024213075060532689),
  ('julho', 0.0024213075060532689),
  ('pela', 0.0024213075060532689),
  ('natalicia', 0.0024213075060532689),
  ('pronunciou', 0.0024213075060532689),
  ('referente', 0.0024213075060532689),
  ('sendo', 0.0024213075060532689),
  ('precederam', 0.0024213075060532689),
  ('afirmara', 0.0024213075060532689),
  ('beverly', 0.0024213075060532689),
  ('respeitosos', 0.0024213075060532689),
  ('tocante', 0.0024213075060532689),
  ('vista', 0.0024213075060532689),
  ('interesses', 0.0024213075060532689),
  ('foi', 0.0024213075060532689),
  ('apreço', 0.0024213075060532689),
  ('aos', 0.0024213075060532689),
  ('ação', 0.0024213075060532689),
  ('embora', 0.0024213075060532689)])

Gerando a distribuição de tópicos para um novo documento:

#### Gera a distribuição de tópicos para um novo documento
Alimenta a base mysql com dados de score (relação) entre tópicos e documentos

In [ ]:
count = 0
#percentil = int(len(cursor.rowcount/100)

pass_sshkey = getpass.getpass()
pass_mysql = getpass.getpass()
with SSHTunnelForwarder(('200.20.164.147', 22),
                        ssh_private_key = "C:/Users/marcelo.ribeiro/Dropbox/A-Marcelo/Educação-Trabalho/2016-CPDOC/Remoto/marcelo_priv_rsa",
                        #ssh_private_key = '/home/rsouza/.ssh/id_rsa',
                        ssh_private_key_password = pass_sshkey,
                        ssh_username="marcelobribeiro",
                        #ssh_username="rsouza",
                        remote_bind_address=('127.0.0.1', 3306)) as server:
    
    conn = pymysql.connect(host='localhost', 
                           port=server.local_bind_port, 
                           user='marcelobribeiro', 
                           #user='rsouza',
                           passwd=pass_mysql,
                           db='CPDOC_AS',
                           use_unicode=True, 
                           charset="utf8")
    cur = conn.cursor()
    '''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
    insere dados na base mysql
    captura documentos da tabela docs, transfere para a tabela topic-doc e insere dados de tópicos
    '''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
    
    cur.execute("DROP TABLE IF EXISTS topic_doc_lang_filter")
    cur.execute('''CREATE TABLE IF NOT EXISTS topic_doc_lang_filter 
               (doc_id VARCHAR(31), topic_id smallint(6), topic_score FLOAT
               DEFAULT NULL)
               ENGINE=MyISAM DEFAULT CHARSET='utf8';''')

    cur.execute("SELECT id,body FROM CPDOC_AS.docs")
    data = cur.fetchall()
    for row in data:
        count += 1 # for break event
        if row is None: break
        text =  row[1]
        text = text.split()
        text_bow = dictionary.doc2bow(text)
        print(lda[text_bow])
        score_list = lda[text_bow]
        doc_id = row[0]
        for score in score_list:
            topic_id = str(score[0])
            topic_score = str(score[1])
            query = "INSERT INTO topic_doc VALUES (%s, %s, %s)"
            cur.execute(query, (doc_id, topic_id, topic_score))
        print(doc_id, topic_id, topic_score)   
        cur.execute("ALTER TABLE CPDOC_AS.topic_doc ORDER BY topic_id ASC, topic_score DESC")
        
        #if count == 10: break #amostra de apenas 10 linhas

#### Visualizando os tópicos com o PyLDAvis

In [ ]:
# https://cran.r-project.org/web/packages/LDAvis/vignettes/details.pdf
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda, corpus, dictionary)

In [ ]:
data_ldavis = pyLDAvis.gensim.prepare(lda, corpus, dictionary)
pyLDAvis.save_html(data_ldavis, os.path.join(outputs,'pyldavis_langfilter_output.html'))